In [1]:
import pandas as pd
import spacy
#import spacy_transformers
import pathlib
import os
#import fr_dep_news_trf
#import openai #!pip install openai
# Il faut aller chercher le token de openai pour utiliser leur API
# L'idée est de constituer un résumé pour chacun des documents et utiliser un moteur de recherche par mot clé sur les textes ?
# Ou sinon uniquement par tag ?

#nlp_max = fr_dep_news_trf.load()
#nlp_eff = spacy.load("fr_core_news_sm")

In [2]:
df = pd.read_csv("./data/data_article_clean.csv")
df

,Unnamed: 0,Unnamed: 0.1,URL,TITRE,IDinDB,TYPE,RESUME,ARTICLE,DATE,YEAR
0,2708,2708,https://www.millenaire3.com/ressources/L-accom...,L'accompagnement des personnes en deuil : qual...,10055,Étude,Au fil des évolutions des sociétés occidentale...,Sommaire : 1 - Besoin d’information et de tran...,2013-09-30,2013
1,1394,1394,https://www.millenaire3.com/ressources/deux-mi...,Deux mille ans de culture à Lyon,10096,Texte,NaN,Lyon n’a connu dans son histoire que quelques ...,2000-01-01,2000
2,1395,1395,https://www.millenaire3.com/ressources/modelis...,Modéliser pour gérer la ville durable,10143,Texte,Article écrit pour la revue M3 n°5\nLa ville d...,"Le projet Vélino’v, démarré en février 2013, i...",2013-01-06,2013
3,1401,1401,https://www.millenaire3.com/ressources/la-vill...,La ville comme système complexe,10184,Texte,Article écrit pour la revue M3 n°5\nLes scienc...,"L’étude des systèmes complexes, interdisciplin...",2013-01-06,2013
4,2061,2061,https://www.millenaire3.com/ressources/le-role...,Le rôle de l’institution hospitalière dans l’u...,10240,Texte,NaN,"Anne-Sophie Clemençon, historienne de l’archit...",2002-01-01,2002
...,...,...,...,...,...,...,...,...,...,...
2769,2770,2770,https://www.millenaire3.com/ressources/2022/sp...,"Spectacles vivants et technologies numériques,...",NaN,Article,"Depuis le siècle dernier, le numérique s’est f...",L’histoire des arts numériques trace un dialog...,2022-01-31,2022
2770,2771,2771,https://www.millenaire3.com/ressources/2022/st...,Stratégie numérique des établissements d’arts ...,NaN,Article,"Site internet, communication sur les réseaux s...",Comment donner sa place au virtuel dans quelqu...,2022-07-02,2022
2771,2776,2776,https://www.millenaire3.com/ressources/2023/je...,"Jeunesse, culture & numérique : les six grands...",NaN,Article,De l’adolescence à l’entrée dans la vie active...,En collaboration avec la direction de la Cultu...,2023-05-05,2023
2772,2777,2777,https://www.millenaire3.com/ressources/danser-...,Danser sur les places : les passions urbaines ...,NaN,Texte,Histoire du danseur Pierre Deloche et de son i...,"Pendant plus de 15 ans, Pierre Deloche a dansé...",2000-01-01,2000


In [10]:
def get_lemma(nlp):

    rows, cols = df.shape 

    for row in range(rows):
        txt = []
        art = str(df.iloc[row].ARTICLE)
        res = str(df.iloc[row].RESUME)
        titre = str(df.iloc[row].TITRE)
        id = df.iloc[row]["Unnamed: 0"]
        
        full = titre+'\n'+res+'\n'+art
        doc2 = nlp(res+'\n'+art)
        
        for token in doc2:
            if token.is_stop == False :#and token.pos_ != "PUNCT":
                txt.append(token.lemma_)
            
        pathlib.Path(f'./data/Texte/{id}').mkdir(parents=True, exist_ok=True)
        with open(f"./data/Texte/{id}/full.txt", "w", encoding="utf8") as f:
            f.write(full)
        with open(f"./data/Texte/{id}/texte.txt", "w", encoding="utf8") as f:
            f.write(" ".join(txt))
        
        print(f"\rSuccessfuly exported {row} / {rows} : {txt[0:10]}", end="")
    
    

In [11]:
# get_lemma(nlp_eff)

Successfuly exported 2773 / 2774 : ['objectif', '2050', ':', 'stratégie', 'national', 'carbone', '(', 'SNBC', ')', ',']nse', 'moyen', 'intégration']']ement']énagement']er']

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import glob
import pandas as pd

In [4]:
df = pd.read_csv("./data/data_article_clean.csv", index_col = 0)
fichiers = glob.glob('data/Texte/*/texte.txt')
nOcc = {}
documents=[]
urls=[]
for fichier in fichiers:
    idx = fichier.split("/")[2]
    url = df[df.index == int(idx)].URL.values[0]
    urls.append(url)
    with open(fichier, encoding='utf8') as f:
        documents.append(f.read())
counter=CountVectorizer()
nOcc=counter.fit_transform(documents)
nOcc=pd.DataFrame.sparse.from_spmatrix(nOcc,index=urls,columns=counter.get_feature_names_out())
nOcc.shape

# tfidfer=TfidfVectorizer(lowercase=False) #On ne maîtrise pas bien la tokennisation de CountVectorizer !
# tfidf_sklearn=tfidfer.fit_transform(documents)
# tfidf_sklearn=pd.DataFrame.sparse.from_spmatrix(tfidf_sklearn,index=urls,columns=tfidfer.get_feature_names_out())




(2774, 53551)

In [7]:
aut_info = pd.read_csv("./pages/data/auteurs_db.csv", index_col=0)

for auteur in aut_info.auteurs.unique():
    print(auteur)
    tmp = aut_info[aut_info.auteurs == auteur].index
    #pathlib.Path(f'./pages/data/Txt/Auteurs/{auteur}').mkdir(parents=True, exist_ok=True)
    if len(tmp) == 1:
        tmp2 = nOcc[nOcc.index == tmp].sum()
    else:
        tmp2 = nOcc[nOcc.index.isin(tmp)].sum()
    tmp2 = tmp2[tmp2 > 0].sort_values(ascending=False)
    tmp2.to_csv(f"./pages/data/Txt/Auteurs/Occurences/{auteur}.csv")


Pierre-Alain FOUR
Caroline BROSSAT
Claire HARPET
Emmanuel ARLOT
Ludovic VIÉVARD
Bertrand LABASSE
Cédric POLÈRE
Miloud LEKOUARA
Stéphane BIENVENUE
Laure BORNAREL
Marianne CHOUTEAU
Annie HONNORAT
Bruno BIGOURDAN
Sandra DECELLE
Emmanuelle LAMBREY
Paul BOINO
Abdel MOKTARI
Geneviève BERNARDIN
Brigitte YVRAY-DANGUIS
Pierre MICHEL_
Julien CASALS
Sylvie  MAURIS-DEMOURIOUX
Catherine PANASSIER
Geoffroy BING
Caroline JANUEL
Florence LE NULZEC
Catherine FORET
Emile HOOGE
Gilles CAYUELA
Valérie PUGIN
Lionel GASTINE
Stéphane AUTRAN
Jean-Loup MOLIN
Boris CHABANEL
Anne-Caroline JAMBAUD
Sylvie  BOUTAUDOU
Isabelle ROUGÉ
Quentin ZARAGORI
Nicolas NOVA
Aurélien BOUTAUD
Didier VANONI
Philippe DUJARDIN
Matthieu BOURY
Pierre GROSDEMOUGE
Pauline KERTUDO
Cécile COULMAIN
Philippe GARGOV
Clément JOURDAN
François MAYSSAL
Natacha ISRAËL
Anouk JORDAN
Frédérique SONNET
Sophie KELLER
Annabelle RICHARD
Théo BATTAGLIA
David CHEVALLIER
Benjamin LIPPENS
Eddy MAAROUFI
Catherine DEBRAND
Nicolas RIO
Benjamin  PRADEL
Julie JE

In [80]:
# tmp3 = tfidf_sklearn[tfidf_sklearn.index.isin(tmp)]
# tmp4 = tmp3.sum() / len(tmp3)

In [88]:
# df2 = pd.DataFrame(tmp4[tmp4 > 0.01].sort_values(ascending=False))
# df2.columns = ["TF-IDF"]
# print(tmp3.sum().sort_values(ascending=False).head(10))
# df2.head(10)

alimentaire    0.951395
fraternité     0.785928
eau            0.651751
fête           0.584658
métropole      0.566700
écologique     0.558371
territoire     0.552630
gastronomie    0.512630
social         0.456172
urbain         0.437399
dtype: float64


,TF-IDF
alimentaire,0.079283
fraternité,0.065494
eau,0.054313
fête,0.048722
métropole,0.047225
écologique,0.046531
territoire,0.046052
gastronomie,0.042719
social,0.038014
urbain,0.036450
